In [1]:
import pandas as pd
import numpy as np
#import matplotlib as plt
import requests
import pdfplumber
import re
#import nltk
import statistics
from urllib.request import Request, urlopen
from io import StringIO, BytesIO

## Pdf urls and pages

In [2]:
abanca={1:{'link':'https://www.abanca.pt', 'page':None}, 
        2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', 'page':6},
        3:{'link':'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf', 'page':6}}

bankinter={1:{'link':'https://www.bankinter.pt', 'page':None},
           2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', 'page':8},
           3:{'link': 'https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf', 'page':None, 'notes':'image file'}}

bic={1:{'link':'https://www.bancobic.ao' ,'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf', 'page':8},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf', 'page':None, 'notes':'wrong file'}}

bai={1:{'link':'https://www.bancobai.ao', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf', 'page':None, 'notes':'No housing credit products'},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf', 'page':None, 'notes':'No housing credit products'}}

ctt={1:{'link':'https://www.bancoctt.pt/', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf', 'page':6},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf', 'page':6}}

## Housing Credit Dictionary

In [3]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

## Prices Scraping Function

In [4]:
def get_prices(pdf,page, dictionary):
    doc = pdfplumber.open(pdf)
    prod_df=pd.DataFrame(doc.pages[page].extract_table())
    
    def find_prices(df):
        for x in range(df.shape[0]):
            for y in range(df.shape[1]):
                if 'Euros' in str(df[y][x]):
                    return y
    
    def find_index(df,dictionary):
        index_col=list()
        for x in range(df.shape[0]):
            for y in range(find_prices(df)):
                if str(df[y][x])=='None' or str(df[y][x])=='' or 'Nota' in str(df[y][x]):
                    pass
                else:
                    words = [word for word in str(df[y][x]).split(' ')]
                    for word in words:
                        for key in dictionary.keys():
                            for item in dictionary[key]:
                                if word in item:
                                    index_col.append(y)

        return int(statistics.mode(index_col))
    
    def clean_df(df):
        for x in range(df.shape[0]):
            for y in df.columns:
                if str(df[y][x])=='' or str(df[y][x])=='None':
                    df[y][x]=np.nan
        return df.dropna(axis='columns', how='all').dropna(axis='rows', how='all')

    
    prices_col=find_prices(prod_df)
    index_col=find_index(prod_df, house_credit_com)
    
    prices_df=pd.DataFrame()
    prices_df['Commissions']=prod_df[index_col]
    prices_df['Prices']=prod_df[prices_col]
    
    return clean_df(prices_df)

def get_pdf(link):
        remote = urlopen(Request(link)).read()
        memory = BytesIO(remote)
        return memory

In [5]:
doc=get_pdf(bankinter[2]['link'])
prices_df=get_prices(doc, 17, house_credit_com)
prices_df.reset_index(drop='index', inplace=True)
prices_df

,Commissions,Prices
0,NaN,Euros \n(min/máx)
1,Avaliações - Fracções \nAutónomas,"250,00"
2,Comissão de Reembolso \nAntecipado Parcial\n- ...,--\n--
3,Comissão por restituição do \nBem fora de praz...,--\n--
4,Comissões relativas a actos \nadministrativos:...,"15,00\n30,00"
5,Comissão pela \nRecuperação de Valores \nem Dí...,"12,00/\n150,00"
6,Rescisões Antecipadas / \nAmortizações Totais\...,--\n--


## Responding function for each subprod

In [6]:
commissions={}
for i in range(len(prices_df)):
    if type(prices_df['Commissions'][i])==str:
        if '\n-' in prices_df['Commissions'][i]:
            names=prices_df['Commissions'][i].split(sep='\n-')
            prices=['']+prices_df['Prices'][i].split(sep='\n')
            for name in names:
                name=name.replace('\n','')
            for price in prices:
                price=price.replace('/','')
                price=price.replace('--','')
            commissions.update({name: price for name, price in zip(names, prices)})
        elif '\n ' in prices_df['Commissions'][i]:
            names=prices_df['Commissions'][i].split(sep='\n ')
            prices=['']+prices_df['Prices'][i].split(sep='\n')
            for name in names:
                name=name.replace('\n','')
            for price in prices:
                price=price.replace('/','')
                price=price.replace('--','')
            commissions.update({name: price for name, price in zip(names, prices)})
        else:
            name = prices_df['Commissions'][i]
            price = prices_df['Prices'][i]
            name = name.replace('\n','')
            name = name.replace('\xa0','')
            price = price.replace('--','')
            price = price.replace('/','')
            commissions.update({name: price})

In [7]:
commissions

{'Avaliações - Fracções Autónomas': '250,00',
 'Comissão de Reembolso \nAntecipado Parcial': '',
 ' Taxa Variável': '--',
 ' Taxa Fixa': '--',
 'Comissão por restituição do \nBem fora de prazo:': '',
 ' Renda Mensal': '--',
 ' Renda Trimestral': '--',
 'Comissões relativas a actos \nadministrativos:': '',
 ' 2ªs vias de facturas / \nrecibos': '15,00',
 ' IMI - Imposto Municipal \nsobre Imóveis e Tarifas de \nSaneamento': '30,00',
 'Comissão pela Recuperação de Valores em Dívida': '12,00\n150,00',
 'Rescisões Antecipadas / \nAmortizações Totais': ''}

In [2]:
from housecredit import Housecredit

IndentationError: unindent does not match any outer indentation level (housecredit.py, line 57)